## 问答关键词提取（进阶方向）
### 任务
- 任务说明：对用户的提问提取关键词
- 任务要求：
    - 计算提问与现有文档的相似度
    - 构造prompt完成意图识别
- 打卡要求：完成RAG完整流程，并提交结果进行打分

文本关键词抽取目标是从给定的文本中提取出最具代表性和有意义的单词或短语。这些关键词通常反映了文本的主题、内容或重要信息。
常见的步骤
- 分词
- 词性标注
- 停用词移除
- 计算词语权重
- 关键词抽取算法 



### IDF

- **分词（Tokenization）**： 将文本拆分为单词或短语。这一步骤将文本转换为基本的语言单元，为后续的处理做准备。

- **移除通用词（Stopword Removal）**： 剔除常见的停用词，如"and"、"the"、"is"等，这些词在文本中普遍出现但往往没有实际的信息价值。这样做可以减少噪音，使关键词更集中在文本的内容性词汇上。

- **计算逆文档频率（IDF）**： 对于每个单词，计算其逆文档频率。逆文档频率是一个衡量单词重要性的指标，它通过对整个文本集合中包含该词的文档数取倒数来计算。

- **计算TF-IDF得分**： 对于每个单词，计算其TF-IDF得分，即词频（TF）与逆文档频率（IDF）的乘积。TF表示单词在当前文档中的出现频率。

- **排序和选取关键词**： 根据计算得到的TF-IDF得分对单词进行排序，选择排名前几的单词作为关键词。排名越高的单词表示在当前文档中具有更高的重要性。

In [3]:
import time 
import jwt
import requests
import jieba
import re
from tqdm import tqdm
import json
import pdfplumber
from langchain.schema import Document
from rank_bm25 import BM25Okapi
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
 

def extract_page_text(filepath, max_len=256, overlap_len=100):
    page_content  = []
    pdf =pdfplumber.open(filepath)
    page_count = 0
    # pattern = r'^\d{1,3}'
    for page in tqdm(pdf.pages):
        page_text = page.extract_text().strip()
        raw_text = [text.strip() for text in page_text.split('\n')]
        new_text = '\n'.join(raw_text)
        new_text = re.sub(r'\n\d{2,3}\s?', '\n', new_text)
        # new_text = re.sub(pattern, '', new_text).strip()
        if len(new_text)>10 and '..............' not in new_text:
            page_content.append(new_text)
        else:
            page_content.append('  ')

    cleaned_chunks = []
    i = 0
    all_str = ''.join(page_content)
    all_str = all_str.replace('\n', '')
    while i<len(all_str):
        cur_s = all_str[i:i+max_len]
        if len(cur_s)>10:
            cleaned_chunks.append(Document(page_content=cur_s, metadata={'page':page_count+1}))
        i+=(max_len - overlap_len)

    return cleaned_chunks,page_content
# 实际KEY，过期时间
def generate_token(apikey: str, exp_seconds: int):
    try:
        id, secret = apikey.split(".")
    except Exception as e:
        raise Exception("invalid apikey", e)

    payload = {
        "api_key": id,
        "exp": int(round(time.time() * 1000)) + exp_seconds * 1000,
        "timestamp": int(round(time.time() * 1000)),
    }
    return jwt.encode(
        payload,
        secret,
        algorithm="HS256",
        headers={"alg": "HS256", "sign_type": "SIGN"},
    )
def ask_glm(content):
    url = "https://open.bigmodel.cn/api/paas/v4/chat/completions"
    headers = {
      'Content-Type': 'application/json',
      'Authorization': generate_token("f1a0b6c3d36d46d3eed74a6c7de3e9e4.pZ88EkbBscyHXXcJ", 1000)
    }

    data = {
        "model": "glm-4",
        "messages": [{"role": "user", "content": content}]
    }

    response = requests.post(url, headers=headers, json=data)
    return response.json()



def get_answer_from_llm(question_idx,questions):
    for query_idx in question_idx:
        doc_scores = bm25.get_scores(jieba.lcut(questions[query_idx]["question"]))
        max_score_page_idxs = doc_scores.argsort()[-3:]

        pairs = []
        for idx in max_score_page_idxs:
            pairs.append([questions[query_idx]["question"], pdf_content[idx] ])

        inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512)
        with torch.no_grad():
            inputs = {key: inputs[key].cuda() for key in inputs.keys()}
            scores = rerank_model(**inputs, return_dict=True).logits.view(-1, ).float()
        max_score_page_idx = max_score_page_idxs[scores.cpu().numpy().argmax()]
        questions[query_idx]['reference'] = 'page_' + str(max_score_page_idx + 1)

        prompt= '''你是一个汽车专家，帮我结合给定的资料，回答所给的问题。如果问题无法从资料中获得，请输出:结合给定的资料，无法回答问题。
    资料：{0}

    问题：{1}
        '''.format(
            ''.join([f'第{i+1}页内容：' + pdf_content[i].replace('\n', '') + '\n' for i in doc_scores.argsort()[-3:]]) ,
            questions[query_idx]["question"]
        )
        prompt2= '''你是一个汽车专家，帮我结合你的经验回答所给的问题.请把回复内容控制在100字以内!
    问题：{0}
        '''.format(
            
            questions[query_idx]["question"]
        )
        answer = ask_glm(prompt)['choices'][0]['message']['content']
        # if '无法回答' in answer:
        #     answer = ask_glm(prompt2)['choices'][0]['message']['content']
        questions[query_idx]['answer'] = answer
        print(query_idx,questions[query_idx])
    return questions

def get_questions_from_file(file_path):
    with open(f'/root/code/submit_task8_glm4.json', 'r', encoding='utf8') as f:
        questions = f.read()
    questions = eval(questions)
    return questions

def get_useless_question_idx(questions):
    ## 收集无法回答答案的问题的索引
    useless_question_idx = []
    for i,question in enumerate(questions):
        if '无法回答' in question['answer']:
            # print(question)
            useless_question_idx.append(i)
    return useless_question_idx

file_path = '/root/code/submit_task8_glm4.json'




questions = json.load(open("./data/questions.json"))
filepath = './data/初赛训练数据集.pdf'
_,pdf_content = extract_page_text(filepath, max_len=256, overlap_len=100)


/root/.conda/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 354/354 [00:06<00:00, 54.43it/s]


In [40]:
import jieba.analyse
import jieba
import re
# jieba.load_userdict('自定义词典.txt')  # 应用自定义词典
jieba.analyse.set_stop_words('stop_words.py')  # 去除自定义停用词

text = '大唐不夜城,不夜城趣味性很高，里面地方特色东西好吃，也有星巴克麦当劳等等选择，有不少场表演，外景夜景一定要薅一个，其它地方很难有这般景象了。娱乐体验了不倒翁，还有十二时辰里面表演更加精彩、内景拍照不错，簋唐楼可以尝试一下沉浸剧本杀……'
text = "".join(re.findall('[\u4e00-\u9fa5]+', text, re.S))
 
# 注意：使用TF-IDF不需要进行分词，直接将原始文本传入使用
jieba.analyse.extract_tags(text,topK=5,withWeight=True)

[('不夜城', 0.7265561498322581),
 ('表演', 0.4472146628148387),
 ('星巴克', 0.4129698504290323),
 ('唐楼', 0.38563766138387096),
 ('不倒翁', 0.3589706106354839)]

In [45]:
def get_idf_keywords(text,top_n=5):
    text = "".join(re.findall('[\u4e00-\u9fa5]+', text, re.S))
    
    # 注意：使用TF-IDF不需要进行分词，直接将原始文本传入使用
    idf_keywords = jieba.analyse.extract_tags(text,topK=top_n,withWeight=True)
    # 只筛选出n名词的单词
    # jieba.analyse.extract_tags(text,topK=5,allowPOS=['n'])
    return idf_keywords

In [46]:
idf_keywords = get_idf_keywords(questions[0]['question'], top_n=5)
print(idf_keywords)

[('第几页', 2.2012550785666667), ('前排', 1.490808432395), ('座椅', 1.3978748176350002), ('通风', 1.3782443116916667), ('内容', 0.8557138958216667)]


### KeyBERT

In [11]:
from keybert import KeyBERT
import jieba 
#模型下载
# from modelscope import snapshot_download
# model_dir = snapshot_download('tiansz/bert-base-chinese',cache_dir='/root/code')

In [32]:
def get_keybert_keywords(text,model,top_n=5 ):
    doc = " ".join(jieba.cut(text))
    keywords = model.extract_keywords(doc, keyphrase_ngram_range=(1,1),  top_n=top_n)
    return keywords
KeyBERT_model = KeyBERT('tiansz/bert-base-chinese')

No sentence-transformers model found with name tiansz/bert-base-chinese. Creating a new one with MEAN pooling.


In [33]:

keybert_keywords = get_keybert_keywords(questions[0]['question'],model=KeyBERT_model, top_n=5)
print(keybert_keywords)

[('第几页', 0.6107), ('座椅', 0.5446), ('前排', 0.4777), ('相关', 0.4681), ('通风', 0.4114)]


### Prompt关键词提取

In [56]:
prompt = """
你是一个专业的文本理解专家，现在请你识别下面内容中的关键词，将关键词使用空格隔开,不要回复其他不相关的内容：

{0}
"""
for query_idx in range(len(questions)):
    question = questions[query_idx]['question']
    prompt_question = prompt.format((question))
    question_keywords = ask_glm(prompt_question)['choices'][0]['message']['content']
    questions[query_idx]['question_keywords'] = question_keywords
    print(questions[query_idx])


{'question': '“前排座椅通风”的相关内容在第几页？', 'answer': '', 'reference': '', 'question_keywords': '前排座椅通风\n'}
{'question': '"关于车辆的儿童安全座椅固定装置，在哪一页可以找到相关内容？"', 'answer': '', 'reference': '', 'question_keywords': '儿童安全座椅 固定装置 相关内容'}
{'question': '“打开前机舱盖”的相关信息在第几页？', 'answer': '', 'reference': '', 'question_keywords': '打开 前机舱盖 相关信息 页码'}
{'question': '“打开前机舱盖”这个操作在哪一页？', 'answer': '', 'reference': '', 'question_keywords': '打开 前机舱盖 操作 页面'}
{'question': '“查看行车记录仪视频”这一项内容在第几页？', 'answer': '', 'reference': '', 'question_keywords': '查看 行车记录仪 视频 第几页'}
{'question': '请问Lynk&Co领克汽车的事件数据记录系统（EDR）主要记录哪些信息？', 'answer': '', 'reference': '', 'question_keywords': '事件数据记录系统 EDR Lynk&Co 领克汽车 记录信息\n'}
{'question': '问题：事件数据记录系统（EDR）中的数据是否可以被黑客利用进行恶意攻击？', 'answer': '', 'reference': '', 'question_keywords': '事件数据记录系统 EDR 数据 黑客 利用 恶意攻击'}
{'question': '问题：在国家环保法要求下，哪些情况下需要对车辆进行报废处理？', 'answer': '', 'reference': '', 'question_keywords': '国家环保法 车辆 报废处理'}
{'question': '请问，如果车辆报废后，原车主是否还能使用该车辆的智能互联服务？', 'answer': '', 'referenc

In [65]:

with open(f'task9_questions.json', 'w', encoding='utf8') as up:
    json.dump(questions, up, ensure_ascii=False, indent=4)

In [ ]:
pdf_contents = [{'content':content,'content_keywords':''} for content in pdf_content]

In [70]:
for query_idx in range(len(pdf_contents[161:])):
    query_idx = query_idx+161
    content = pdf_contents[query_idx]['content']
    prompt_content = prompt.format((content))
    content_keywords = ask_glm(prompt_content)['choices'][0]['message']['content']
    pdf_contents[query_idx]['content_keywords'] = content_keywords
    print(pdf_contents[query_idx])
    # break

# with open(f'task9_pdf_contents.json', 'w', encoding='utf8') as up:
#     json.dump(pdf_contents, up, ensure_ascii=False, indent=4)

{'content': '启动和驾驶\n加油 注意！\n警告！ ■ 防止燃油溅出，否则可能会损坏车辆表面涂漆。\n■ 燃油具有高度易燃性。如果您不正确处理燃油，则存在火灾和爆 ■ 请务必注入满足规格要求的燃油，避免损坏车辆。\n炸的风险。\n■ 必须远离火源、明火，不得产生火花，亦不得抽烟。加油前，关\n闭发动机。\n加油\n■ 避免吸入燃油蒸气或让燃油溅到您的皮肤或眼睛。让儿童远离燃\n油。 1 打开燃油加注口盖板前，车辆应处于熄火且解锁的状态。\n■ 静电的积聚会产生火花并点燃燃油蒸汽，因此存在火灾和爆炸的\n2短按仪表板上左侧的油箱盖板开启按钮 。\n风险。\n■ 在接触加油枪之前，务必先接触远离加油管口、加油枪或其他燃\n油来源的车辆的其他金属部位。这样便可以释放任何积聚的静\n电。\n■ 加油时不要使用手机。手机信号会造成静电积聚并点燃燃油蒸\n汽，造成火灾和人身伤害。\n警告！\n加油时应遵守以下注意事项：\n■ 将加油枪牢牢地插入加油管口中。\n■ 在完成加油以及关好燃油加注口盖板之前，不得启动车辆。\n■ 在加油枪自动发出咔嗒声后，停止注油。\n3轻按燃油加注口盖板的后部，将其打开。\n警告！\n如果您或其他人需接触燃油，应遵守以下规定：\n■ 立即将皮肤上的燃油清洗干净。\n■ 如果燃油溅入眼睛，使用清洁的水将眼睛彻底清洗，并立即寻求\n医疗救助。\n', 'content_keywords': '启动 驾驶 加油 警告 燃油溅出 损坏车辆表面涂漆 易燃性 火灾 爆炸 注入规格要求燃油 火源 明火 火花 抽烟 关闭发动机 吸入燃油蒸气 皮肤 眼睛 儿童 打开燃油加注口盖板 熄火 解锁 静电 积聚 加油枪 油箱盖板开启按钮 金属部位 静电积聚 手机信号 火灾 人身伤害 插入加油管口 完成加油 燃油加注口盖板 启动车辆 咔嗒声 注油 打开 加油规定 清洗 皮肤 燃油 眼睛 医疗救助'}
{'content': '启动和驾驶\n辅助加油工具\n当车辆需要手动加注燃油时，可以通过车辆附带的加油漏斗进行添\n加。\n4将加油枪直接插入燃油加注口，开始加油。\n加油漏斗\n1 从后备厢的存放区域中取出加油漏斗。\n2打开燃油加注口盖板。\n3将加油漏斗插入燃油加注口，开始加油。\n警告！\n■ 通过漏斗加注燃油时，应注意观察漏斗内液面情况，避免燃\n油洒出车外

In [74]:
with open(f'task9_pdf_contents.json', 'w', encoding='utf8') as up:
    json.dump(pdf_contents, up, ensure_ascii=False, indent=4)

### 文本聚类

In [8]:
# !pip install text2vec umap-learn 

import torch
from sklearn.cluster import MiniBatchKMeans
import pandas as pd
from text2vec import SentenceModel
from sklearn.metrics import silhouette_score
import numpy as np
import umap  

In [ ]:
def read_xlsx(data_path):
    print("正在读取源文档...")
    df = pd.read_excel(data_path, header=None)
    sentences = df[0].tolist()
    sentences = list(set([str(i).strip() for i in sentences]))
    return sentences  

In [13]:
with open(f'task9_pdf_contents.json', 'r', encoding='utf8') as up:
    pdf_contents = eval(up.read())
with open(f'task9_questions.json', 'r', encoding='utf8') as up:
    pdf_questions = eval(up.read())

In [14]:
def text2vec(sentences):
    print("正在将文本转换为向量，请耐心等待...")
    model = SentenceModel("xrunda/m3e-base")
    embeddings = []
    for i in sentences:
        embedding = model.encode(i)
        embeddings.append(embedding)
    umap.UMAP().fit_transform(embeddings) # 降维
    embeddings_array = np.array(embeddings)
    x = torch.from_numpy(embeddings_array)
    return x 

In [17]:
def search_k_value(sentences, k_min=2, k_max=10):
    """
    :参数 data_path: 待聚类的文本，在xlsx中为一列，且列名为句子

    :输出 k: 在终端中打印 k 值所对应的轮廓系数，最终选择轮廓系数最大的 k 值
    """

    x = text2vec(sentences)
    print("正在搜索最佳 k 值，请耐心等待...")
    # 循环计算不同的 K 值下的轮廓系数
    for k in range(k_min, k_max + 1):
        # 创建 K-means 聚类模型
        mbk = MiniBatchKMeans(n_clusters=k)
        mbk = mbk.fit(x)
        cluster_ids_x = mbk.labels_
        # 计算轮廓系数
        silhouette_avg = silhouette_score(x, cluster_ids_x)
        print(f"K={k}, 轮廓系数: {silhouette_avg}") 

In [19]:
def clustering(sentences, k=10):
    # sentences = read_xlsx(data_path)
    x = text2vec(sentences)
    print("正在将向量进行聚类，请耐心等待...")
    # 创建 K-means 聚类模型
    mbk = MiniBatchKMeans(n_clusters=k)
    mbk = mbk.fit(x)
    cluster_ids_x = mbk.labels_
    output_df = pd.DataFrame()
    output_df["句子"] = sentences
    output_df["类别"] = cluster_ids_x
    output_df = output_df.sort_values(by="类别", ascending=False)
    # output_df.to_excel("output.xlsx", index=False) 
    return output_df


In [16]:
docs = [pdf_content['content'] for pdf_content in pdf_contents]
search_k_value(docs, k_min=2, k_max=10)

In [20]:
output_df = clustering(docs, k=10)

正在将文本转换为向量，请耐心等待...


2024-02-16 22:47:06.488 | DEBUG    | text2vec.sentence_model:__init__:80 - Use device: cuda


正在将向量进行聚类，请耐心等待...


In [21]:
output_df

,句子,类别
334,紧急情况下\n说明！ 插入机械钥匙。\n转动钥匙至一定角度后，拔出机械钥匙并放平车门外把手，...,9
335,紧急情况下\n应急解锁充电枪\n当车辆充电时发生突发状况（如车辆整车断电、电子锁机械故障），...,9
337,紧急情况下\n牵引车辆 车辆牵引至平板拖车前，先将变速器挂入空挡（N）并解除EPB功\n能。...,9
320,保养和维护\n下表列举了您的爱车所需的常规保养项目。此外，您的维修技师可能 特别恶劣的行驶条...,9
307,保养和维护\n2抬起前机舱盖。 3轻抬机舱盖前边缘，检查确保其已完全关闭。\n警告！ 警告！...,9
...,...,...
231,泊车\n说明！ RPA主要包含以下功能：\n■ 直线遥控功能。\n□ 完成泊车后，您可能还需...,0
232,泊车\n说明\n长按 控制车辆向前直线行驶。\n□ RPA工作时车辆会自动执行转向和制动操作...,0
233,泊车\n3选择停车位：在中央显示屏上选择需要泊入的停车位，选中后该\n停车位高亮显示。\n4...,0
234,泊车\n5泊车完成：Lynk&CoApp上会提示泊车完成，此时驾驶员可以接\n点击选择垂直泊...,0


### 参考资料
- [KeyBert关键词提取 ：原理、方法介绍、代码实践](https://blog.csdn.net/chenhepg/article/details/118571671)

- [如何对大量文本进行聚类？](https://www.bilibili.com/read/cv25474320/)